In [2]:
# Importing Libraries
from sklearnex import patch_sklearn #Improves sklearn alghoritms performance
patch_sklearn()
import sklearn
print('scikit-learn version\n', sklearn.__version__)
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict
from aux_functions.name_2lines import name_2lines

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


scikit-learn version
 1.0.2


In [3]:
#Set seed
seed=41

In [4]:
classif_level = 3

# Load Data

In [5]:
feature_names = pickle.load(open('Dataset/feature_names.pkl', 'rb'))
X_train = pickle.load(open(f'Dataset/X_train.pkl', 'rb'))
X_train_scal = pickle.load(open(f'Dataset/X_train_scal.pkl', 'rb'))
y_train = pickle.load(open(f'Dataset/y_train.pkl', 'rb'))
#all classes
classes = pickle.load(open('Dataset/all_categories.pkl', 'rb'))['Class']
#Classes with unique child
c_u = pickle.load(open(f'Dataset/C_wunique_child.pkl', 'rb'))

In [6]:
for i, class_ in enumerate(classes):
    print(i, class_)

0 (3'->5')-dinucleotides and analogues
1 (5'->5')-dinucleotides
2 2-arylbenzofuran flavonoids
3 5'-deoxyribonucleosides
4 6,7-benzomorphans
5 Acetylides - Unspecified
6 Acyclic allenes
7 Acyl halides
8 Ajmaline-sarpagine alkaloids
9 Alkali metal organides
10 Alkali metal oxoanionic compounds
11 Alkali metal salts
12 Alkaline earth metal organides
13 Alkaline earth metal oxoanionic compounds
14 Alkaline earth metal salts
15 Alkaloids and derivatives - Unspecified
16 Alkyl halides
17 Allyl sulfur compounds
18 Allyl-type 1,3-dipolar organic compounds
19 Alpha-methyldeoxybenzoin flavonoids
20 Amaryllidaceae alkaloids
21 Angucyclines
22 Anthracenes
23 Anthracyclines
24 Aporphines
25 Aryl halides
26 Arylnaphthalene lignans
27 Aryltetralin lignans
28 Aspidofractine alkaloids
29 Aspidospermatan-type alkaloids
30 Aurone flavonoids
31 Azacyclic compounds
32 Azaphilones
33 Azaspirodecane derivatives
34 Azepanes
35 Azepines
36 Azetidines
37 Azobenzenes
38 Azoles
39 Azolidines
40 Azolines
41 Benzaz

## Train and Tune classifiers

### Random Forests

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        print('Only one subchild')
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Subclass'][class_]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train[y_train[:, classif_level-1]==class_][:, f_index]
            grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
                'criterion' : ['gini', 'entropy'],
                'max_features' : range(1, len(sel_f)+1)}
        else:
            X_train_ = X_train[y_train[:, classif_level-1]==class_]
            grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
                'criterion' : ['gini', 'entropy'],
                'max_features' : [1, 2, 3, 5, 7, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 133]}
        y_train_ = y_train[y_train[:, classif_level-1]==class_][:, classif_level]
        
        estimator = RandomForestClassifier(random_state=seed, n_jobs=-1, n_estimators=200)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Subclass/{i}_RF_{f_sel}_feat.pkl', 'wb'))

In [19]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Subclass/{i}_RF_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

(3'->5')-dinucleotides and analogues
(5'->5')-dinucleotides
2-arylbenzofuran flavonoids
5'-deoxyribonucleosides
all
f1_macro_val -> 0.924001924001924
f1_macro_train -> 1.0
f1_micro_val -> 0.9259259259259259
f1_micro_train -> 1.0
sel
f1_macro_val -> 0.924001924001924
f1_macro_train -> 1.0
f1_micro_val -> 0.9259259259259259
f1_micro_train -> 1.0
6,7-benzomorphans
Acetylides - Unspecified
Acyclic allenes
Acyl halides
Ajmaline-sarpagine alkaloids
Alkali metal organides
Alkali metal oxoanionic compounds
all
f1_macro_val -> 0.9240669240669241
f1_macro_train -> 1.0
f1_micro_val -> 0.9674603174603175
f1_micro_train -> 1.0
sel
f1_macro_val -> 0.9240669240669241
f1_macro_train -> 1.0
f1_micro_val -> 0.9674603174603175
f1_micro_train -> 1.0
Alkali metal salts
all
f1_macro_val -> 1.0
f1_macro_train -> 1.0
f1_micro_val -> 1.0
f1_micro_train -> 1.0
sel
f1_macro_val -> 1.0
f1_macro_train -> 1.0
f1_micro_val -> 1.0
f1_micro_train -> 1.0
Alkaline earth metal organides
Alkaline earth metal oxoanionic co

f1_macro_val -> 0.6342158940117817
f1_macro_train -> 0.7692670631574017
f1_micro_val -> 0.7939393939393941
f1_micro_train -> 0.8878787878787878
Diazanaphthalenes
all
f1_macro_val -> 0.6504329248491686
f1_macro_train -> 0.9783634549164303
f1_micro_val -> 0.929915698336751
f1_micro_train -> 0.9936280275433648
sel
f1_macro_val -> 0.6607092389683001
f1_macro_train -> 0.9799717676277705
f1_micro_val -> 0.9299156983367509
f1_micro_train -> 0.994425475709234
Diazinanes
Diazines
all
f1_macro_val -> 0.5949923699336354
f1_macro_train -> 0.9706228745459327
f1_micro_val -> 0.8130081300813008
f1_micro_train -> 0.9813008130081301
sel
f1_macro_val -> 0.5886850947091885
f1_macro_train -> 0.9688456949325719
f1_micro_val -> 0.8065040650406504
f1_micro_train -> 0.9796747967479674
Dibenzocycloheptenes
Dibenzylbutane lignans
all
f1_macro_val -> 0.7277777777777777
f1_macro_train -> 1.0
f1_micro_val -> 0.7619047619047619
f1_micro_train -> 1.0
sel
f1_macro_val -> 0.8444444444444444
f1_macro_train -> 0.9477541

f1_macro_train -> 1.0
f1_micro_val -> 0.9841269841269842
f1_micro_train -> 1.0
sel
f1_macro_val -> 0.9621621621621621
f1_macro_train -> 1.0
f1_micro_val -> 0.9841269841269842
f1_micro_train -> 1.0
Nucleoside and nucleotide analogues
all
f1_macro_val -> 0.9575424575424575
f1_macro_train -> 1.0
f1_micro_val -> 0.96875
f1_micro_train -> 1.0
sel
f1_macro_val -> 0.9437044437044437
f1_macro_train -> 1.0
f1_micro_val -> 0.95864898989899
f1_micro_train -> 1.0
Nucleosides, nucleotides, and analogues - Unspecified
Organic acids and derivatives - Unspecified
Organic carbonic acids and derivatives
all
f1_macro_val -> 0.7985569042431787
f1_macro_train -> 0.98805256869773
f1_micro_val -> 0.8608736942070275
f1_micro_train -> 0.9935897435897436
sel
f1_macro_val -> 0.8140330947193691
f1_macro_train -> 0.9895506792058516
f1_micro_val -> 0.8732193732193733
f1_micro_train -> 0.9935897435897436
Organic compounds - Unspecified
Organic disulfides
all
f1_macro_val -> 0.9092436974789916
f1_macro_train -> 1.0
f

f1_macro_val -> 0.3851444851444852
f1_macro_train -> 0.9687552213868004
f1_micro_val -> 0.49027777777777776
f1_micro_train -> 0.9791666666666666
Pyrenes
all
f1_macro_val -> 0.9331619802634297
f1_macro_train -> 0.9733279978658129
f1_micro_val -> 0.9333333333333332
f1_micro_train -> 0.9733333333333333
sel
f1_macro_val -> 0.9196137330195301
f1_macro_train -> 0.9733279978658129
f1_micro_val -> 0.9199999999999999
f1_micro_train -> 0.9733333333333333
Pyridines and derivatives
all
f1_macro_val -> 0.3834666215134033
f1_macro_train -> 0.9606529024376718
f1_micro_val -> 0.5447763180662594
f1_micro_train -> 0.9636434232540777
sel
f1_macro_val -> 0.3944700920558823
f1_macro_train -> 0.9606409252716984
f1_micro_val -> 0.5489923997391153
f1_micro_train -> 0.9636434232540777
Pyridopyrimidines
all
f1_macro_val -> 0.5907814407814408
f1_macro_train -> 1.0
f1_micro_val -> 0.6444444444444445
f1_micro_train -> 1.0
sel
f1_macro_val -> 0.5836619652748684
f1_macro_train -> 0.9907968574635241
f1_micro_val -> 0

### KNN

In [ ]:
grid = {'n_neighbors': range(1, 31), 'weights' : ['uniform', 'distance'],
        'metric':['manhattan', 'euclidean', 'cosine']}

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        print('Only one subchild')
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        grid['n_neighbors'] = range(1, 31)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Subclass'][class_]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_]
            
        y_train_ = y_train[y_train[:, classif_level-1]==class_][:, classif_level]
        n_samples = int(len(y_train_)*2/3)
        if n_samples<30:
            grid['n_neighbors'] = range(1, n_samples + 1)
        estimator = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=10)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Subclass/{i}_KNN_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Subclass/{i}_KNN_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### Logistic Regression

In [ ]:
grid = [{'penalty': ['l1'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['saga']},
        {'penalty': ['elasticnet'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'l1_ratio': [0.25, 0.5, 0.75],
         'solver': ['saga']},
        {'penalty': ['l2'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['sag', 'saga']},
       {'penalty': ['none'],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['sag', 'saga']}]

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        print('Only one subchild')
        continue
    if i in [42, 74, 103, 111, 112]:
        max_iter = 100
    else:
        max_iter = 5000
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Subclass'][class_]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_]
            
        y_train_ = y_train[y_train[:, classif_level-1]==class_][:, classif_level]
        
        estimator = LogisticRegression(max_iter=max_iter, tol=1e-6, random_state=seed, n_jobs=-1)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Subclass/{i}_LR_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Subclass/{i}_LR_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### SVM

In [ ]:
grid = [{'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10, 100],
        'class_weight': [None, 'balanced']}]

In [ ]:
for i, class_ in enumerate(classes):
    if class_ in c_u:
        print('Only one subchild')
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Subclass'][class_]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_]
            
        y_train_ = y_train[y_train[:, classif_level-1]==class_][:, classif_level]
        
        estimator = LinearSVC(dual=False, loss='squared_hinge', multi_class='ovr', tol=1e-6, max_iter=5000, 
                              random_state=seed, verbose=3)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Subclass/{i}_SVM_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Subclass/{i}_SVM_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### ComplementNB

In [ ]:
grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 
        'norm': [True, False]}

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        print('Only one subchild')
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Subclass'][class_]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==class_]
            
        y_train_ = y_train[y_train[:, classif_level-1]==class_][:, classif_level]
        
        estimator = ComplementNB()
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Subclass/{i}_NB_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, class_ in enumerate(classes):
    print(class_)
    if class_ in c_u:
        continue
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Subclass/{i}_NB_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### Load GS scores

In [11]:
scores_df = []
params_df = []
for i, class_ in enumerate(classes):
    if class_ in c_u:
        continue
    #Select if result is better with or without f selection
    scores_rows = [{'Classifier':class_, 'F1-score average': 'macro'}, {'Classifier':class_, 'F1-score average': 'micro'}]
    params_row = {'Classifier':class_}
    for alg in ['RF', 'KNN', 'LR', 'SVM', 'NB']:
        f_sel_gs = {}
        for f_sel in ['all', 'sel']:
            f_sel_gs[f_sel] = pickle.load(open(f'Models/Subclass/{i}_{alg}_{f_sel}_feat.pkl', 'rb'))
        score_all = f_sel_gs['all'].cv_results_[f'mean_test_f1_macro'][f_sel_gs['all'].best_index_]
        score_sel = f_sel_gs['sel'].cv_results_[f'mean_test_f1_macro'][f_sel_gs['sel'].best_index_]
        if score_all >= score_sel:
            f_sel = 'all'
            f = 'No'
        else:
            f_sel = 'sel'
            f = 'Yes'
        gs = f_sel_gs[f_sel]
        #Save scores
        scores_rows[0][f'{alg} (val)'] = gs.cv_results_[f'mean_test_f1_macro'][gs.best_index_]
        scores_rows[0][f'{alg} (train)'] = gs.cv_results_[f'mean_train_f1_macro'][gs.best_index_]
        scores_rows[1][f'{alg} (val)'] = gs.cv_results_[f'mean_test_f1_micro'][gs.best_index_]
        scores_rows[1][f'{alg} (train)'] = gs.cv_results_[f'mean_train_f1_micro'][gs.best_index_]
        #Save parameters
        params = gs.cv_results_['params'][gs.best_index_]
        params_row[f'{alg} (Feature Selection)'] = f
        for param_name, param in params.items():
            params_row[f'{alg} ({param_name})'] = param
    scores_df.extend(scores_rows)
    params_df.append(params_row)    
scores_df = pd.DataFrame(scores_df).set_index(['Classifier', 'F1-score average'])
params_df = pd.DataFrame(params_df).set_index(['Classifier'])

Exception ignored in: <function ZipFile.__del__ at 0x7f294aeb61f0>
Traceback (most recent call last):
  File "/home/hsilva/anaconda3/lib/python3.9/zipfile.py", line 1816, in __del__
    self.close()
  File "/home/hsilva/anaconda3/lib/python3.9/zipfile.py", line 1833, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
Exception ignored in: <function ZipFile.__del__ at 0x7f294aeb61f0>
Traceback (most recent call last):
  File "/home/hsilva/anaconda3/lib/python3.9/zipfile.py", line 1816, in __del__
    self.close()
  File "/home/hsilva/anaconda3/lib/python3.9/zipfile.py", line 1834, in close
    self._write_end_record()
  File "/home/hsilva/anaconda3/lib/python3.9/zipfile.py", line 1928, in _write_end_record
    self.fp.write(endrec)
io.UnsupportedOperation: write


In [12]:
with pd.ExcelWriter('Results/GS_scores.xlsx', mode='a') as writer:  
    scores_df.to_excel(writer, sheet_name='Subclass')
scores_df

ValueError: Sheet 'Subclass' already exists and if_sheet_exists is set to 'error'.

In [ ]:
with pd.ExcelWriter('Results/GS_best_comb_params.xlsx', mode='a') as writer:  
    params_df.to_excel(writer, sheet_name='Subclass')
params_df

In [13]:
scores_df

RF (val)  RF (train)  \
Classifier                        F1-score average                         
5'-deoxyribonucleosides           macro             0.924002    1.000000   
                                  micro             0.925926    1.000000   
Alkali metal oxoanionic compounds macro             0.924067    1.000000   
                                  micro             0.967460    1.000000   
Alkali metal salts                macro             1.000000    1.000000   
...                                                      ...         ...   
Tropones                          micro             0.633333    0.969697   
Unsaturated hydrocarbons          macro             0.655582    0.822103   
                                  micro             0.786461    0.895205   
Vinyl halides                     macro             0.956863    1.000000   
                                  micro             0.969697    1.000000   

                                                    KNN (val)  KNN (train)  \
Classifier                        F1-score average                           
5'-deoxyribonucleosides           macro              0.953846     0.349567   
                                  micro              0.962963     0.407407   
Alkali metal oxoanionic compounds macro              0.847876     1.000000   
                                  micro              0.934127     1.000000   
Alkali metal salts                macro              0.791667     0.932660   
...                                                       ...          ...   
Tropones                          micro              0.766667     0.751515   
Unsaturated hydrocarbons          macro              0.675302     0.841020   
                                  micro              0.774697     0.895205   
Vinyl halides                     macro              1.000000     1.000000   
                                  micro              1.000000     1.000000   

                                                    LR (val)  LR (train)  \
Classifier                        F1-score average                         
5'-deoxyribonucleosides           macro             0.924002    0.923385   
                                  micro             0.925926    0.925926   
Alkali metal oxoanionic compounds macro             0.786910    0.896446   
                                  micro             0.870635    0.935540   
Alkali metal salts                macro             1.000000    1.000000   
...                                                      ...         ...   
Tropones                          micro             0.822222    0.845455   
Unsaturated hydrocarbons          macro             0.534575    0.556779   
                                  micro             0.688095    0.721119   
Vinyl halides                     macro             1.000000    1.000000   
                                  micro             1.000000    1.000000   

                                                    SVM (val)  SVM (train)  \
Classifier                        F1-score average                           
5'-deoxyribonucleosides           macro              0.924002     0.943452   
                                  micro              0.925926     0.944444   
Alkali metal oxoanionic compounds macro              0.786910     0.886541   
                                  micro              0.870635     0.927216   
Alkali metal salts                macro              0.725000     1.000000   
...                                                       ...          ...   
Tropones                          micro              0.766667     0.784848   
Unsaturated hydrocarbons          macro              0.423081     0.446907   
                                  micro              0.545518     0.600838   
Vinyl halides                     macro              1.000000     1.000000   
                                  micro              1.000000     1.000000   

                                            

In [14]:
%matplotlib inline
sns.set_style('white')
fig, ax = plt.subplots(figsize=(5, 22), dpi=800)
sns.heatmap(scores_df[:58], cmap="Blues", ax=ax, cbar_kws={'shrink': .4, 'pad':0.02})

ylabel_mapping = OrderedDict()
for name, average in scores_df[:58].index:
    ylabel_mapping.setdefault(name, [])
    ylabel_mapping[name].append(average)

hline = []
averages_labels = []
names = []
for name, average_list in ylabel_mapping.items():
    averages_labels.extend(average_list)
    names.append(name)
    if hline:
        hline.append(len(average_list) + hline[-1])
    else:
        hline.append(len(average_list))

ax.hlines(hline, xmin=-1, xmax=10, color="white", linewidth=3)
ax.set_yticklabels(averages_labels, fontdict={'fontsize':9})
for y, name in enumerate(name_2lines(names, 15)):
    plt.text(-1.7, y*2+1, name, ha='right', va='center', fontsize=12)
ax.set_ylabel('Classifier (f1-score average)', labelpad=150)

ax.set_xticklabels(['val', 'train', 'val', 'train', 'val', 'train', 'val', 'train', 'val', 'train'], rotation=0)
for x, name in enumerate(['RF', 'KNN', 'LR', 'SVM', 'NB']):
    plt.text(x*2+1, 59.5, name, ha='center', va='center', fontsize=11)
    
plt.savefig(f'Plots/GS scores (Subclass) heatmap_1', bbox_inches='tight', transparent=True, dpi=800)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 22), dpi=800)
sns.heatmap(scores_df[58:116], cmap="Blues", ax=ax, cbar_kws={'shrink': .4, 'pad':0.02})

ylabel_mapping = OrderedDict()
for name, average in scores_df[58:116].index:
    ylabel_mapping.setdefault(name, [])
    ylabel_mapping[name].append(average)

hline = []
averages_labels = []
names = []
for name, average_list in ylabel_mapping.items():
    averages_labels.extend(average_list)
    names.append(name)
    if hline:
        hline.append(len(average_list) + hline[-1])
    else:
        hline.append(len(average_list))

ax.hlines(hline, xmin=-1, xmax=10, color="white", linewidth=3)
ax.set_yticklabels(averages_labels, fontdict={'fontsize':9})
for y, name in enumerate(name_2lines(names, 15)):
    plt.text(-1.7, y*2+1, name, ha='right', va='center', fontsize=12)
ax.set_ylabel('Classifier (f1-score average)', labelpad=150)

ax.set_xticklabels(['val', 'train', 'val', 'train', 'val', 'train', 'val', 'train', 'val', 'train'], rotation=0)
for x, name in enumerate(['RF', 'KNN', 'LR', 'SVM', 'NB']):
    plt.text(x*2+1, 59.5, name, ha='center', va='center', fontsize=11)
    
plt.savefig(f'Plots/GS scores (Subclass) heatmap_2', bbox_inches='tight', transparent=True, dpi=800)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 22), dpi=800)
sns.heatmap(scores_df[116:174], cmap="Blues", ax=ax, cbar_kws={'shrink': .4, 'pad':0.02})

ylabel_mapping = OrderedDict()
for name, average in scores_df[116:174].index:
    ylabel_mapping.setdefault(name, [])
    ylabel_mapping[name].append(average)

hline = []
averages_labels = []
names = []
for name, average_list in ylabel_mapping.items():
    averages_labels.extend(average_list)
    names.append(name)
    if hline:
        hline.append(len(average_list) + hline[-1])
    else:
        hline.append(len(average_list))

ax.hlines(hline, xmin=-1, xmax=10, color="white", linewidth=3)
ax.set_yticklabels(averages_labels, fontdict={'fontsize':9})
for y, name in enumerate(name_2lines(names, 15)):
    plt.text(-1.7, y*2+1, name, ha='right', va='center', fontsize=12)
ax.set_ylabel('Classifier (f1-score average)', labelpad=150)

ax.set_xticklabels(['val', 'train', 'val', 'train', 'val', 'train', 'val', 'train', 'val', 'train'], rotation=0)
for x, name in enumerate(['RF', 'KNN', 'LR', 'SVM', 'NB']):
    plt.text(x*2+1, 59.5, name, ha='center', va='center', fontsize=11)
    
plt.savefig(f'Plots/GS scores (Subclass) heatmap_3', bbox_inches='tight', transparent=True, dpi=800)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 22), dpi=800)
sns.heatmap(scores_df[174:], cmap="Blues", ax=ax, cbar_kws={'shrink': .4, 'pad':0.02})

ylabel_mapping = OrderedDict()
for name, average in scores_df[174:].index:
    ylabel_mapping.setdefault(name, [])
    ylabel_mapping[name].append(average)

hline = []
averages_labels = []
names = []
for name, average_list in ylabel_mapping.items():
    averages_labels.extend(average_list)
    names.append(name)
    if hline:
        hline.append(len(average_list) + hline[-1])
    else:
        hline.append(len(average_list))

ax.hlines(hline, xmin=-1, xmax=10, color="white", linewidth=3)
ax.set_yticklabels(averages_labels, fontdict={'fontsize':9})
for y, name in enumerate(name_2lines(names, 15)):
    plt.text(-1.7, y*2+1, name, ha='right', va='center', fontsize=12)
ax.set_ylabel('Classifier (f1-score average)', labelpad=150)

ax.set_xticklabels(['val', 'train', 'val', 'train', 'val', 'train', 'val', 'train', 'val', 'train'], rotation=0)
for x, name in enumerate(['RF', 'KNN', 'LR', 'SVM', 'NB']):
    plt.text(x*2+1, 59.5, name, ha='center', va='center', fontsize=11)
    
plt.savefig(f'Plots/GS scores (Subclass) heatmap_4', bbox_inches='tight', transparent=True, dpi=800)
plt.show()